In [ ]:
# 패키지 설치
!pip install yfinance --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
sns.set()

from scipy.stats import norm, uniform
import random

## 데이터셋 준비

In [ ]:
# 다섯 종목에 대한 데이터
tickers = ['GOOGL', 'AMZN', 'AAPL', 'MSFT', 'NFLX']

stocks = yf.Tickers(tickers)

data = stocks.history(start='2000-01-01', actions=False)

data.drop(['Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)

data = data.droplevel(0, axis=1)

In [ ]:
data.plot()

In [ ]:
rets = data.pct_change().fillna(0)
rets.plot()

### 단일 자산에 대한 몬테카를로 시뮬레이션 VaR, CVaR

In [ ]:
# T 시점 후의 가격 계산 함수
def terminal_price(S_0, mu, sigma, Z, T):
    return S_0 * np.exp((mu - sigma**2 / 2) * T + sigma * np.sqrt(T) * Z)

In [ ]:
# 애플 주가 데이터
aapl = data['AAPL']

# 애플 수익률
aapl_rets = aapl.pct_change().dropna()

# 애플 평균 수익률
mu = aapl_rets.mean() * 252

# 애플 표준편차
sigma = aapl_rets.std() * np.sqrt(252)

# 시뮬레이션 기간 (3개월)
T = 0.25

# 시뮬레이션 횟수
n_simulations = 10000

# VaR %
delta = 0.01

# 현재 주가
S_0 = aapl.iloc[-1]

In [ ]:
# 난수 생성
Z = norm.rvs(size=[1, n_simulations])

# 주가 미래 가격에 대한 시뮬레이션
S_T = pd.Series(terminal_price(S_0, 0, sigma, Z, T).flatten())

# 포트폴리오 수익률 계산
aapl_return = (S_T - S_0) / S_0

In [ ]:
# VaR
aapl_VaR = aapl_return.quantile(delta)

# CVaR
aapl_CVaR = aapl_return[aapl_return <= aapl_VaR].mean()

In [ ]:
print('AAPL VaR = {}'.format(np.round(aapl_VaR * 100, 2)))
print('AAPL CVaR = {}'.format(np.round(aapl_CVaR * 100, 2)))

In [ ]:
# VaR, CVaR 그래프
plt.figure(figsize=(10, 8))
plt.hist(aapl_return, bins=100)
plt.title('Monte Carlo Simulation')
plt.axvline(aapl_VaR, color='g', linewidth=3, linestyle='dashed', label='VaR={}'.format(np.round(aapl_VaR, 2)))
plt.axvline(aapl_CVaR, color='r', linewidth=3, linestyle='dashed', label='CVaR={}'.format(np.round(aapl_CVaR, 2)))
plt.legend()

### 시뮬레이션을 위한 초기 패러미터값 설정

In [ ]:
# Parameters

# 초기 주가
S_0 = data.iloc[-1]

# 수익률
mu = rets.mean() * 252

# 변동성
sigma = rets.std() * np.sqrt(252)

# 상관계수 행렬
corr = rets.corr()

# 촐레스키 분해
L = np.linalg.cholesky(corr)

# 시뮬레이션 기간 (1년)
T = 1

### 촐레스키 분해

$$ A = LL^{T} $$

- 촐레스키 분해

어떤 주어진 행렬을 위삼각행렬과 아랫삼각행렬로 분해하는 기법.

윗삼각행렬과 아랫삼각행렬은 서로 대칭적임.

- 촐레스키 분해를 사용하는 이유

몬테카를로 시뮬레이션을 실행할 때 주가간에 존재하는 상관관계를 반영하여 주가의 미래 경로를 생성하기 위함.

즉, 서로 상관관계가 존재하는 난수를 생성할 수 있게 해주는 도구가 바로 촐레스키 분해임.


### 몬테카를로 시뮬레이션 VaR

In [ ]:
# 난수 생성 시드 = 0
np.random.seed(0)

# 시뮬레이션 횟수
n_simulations = 10000

# VaR %
delta = 0.01

# 현재 포트폴리오 가치
portval_current = np.sum(S_0)
print(portval_current)

### 기하 브라운 운동: 자산 가격 다이나믹스

$$ S_{T}= S_{0}e^{(r-\frac{\sigma^{2}}{2})*T+\sigma\sqrt{T}Z} $$

- $S_{T}$ : T 시점 이후 주가
- $S_{0}$ : 현재 주가
- $r$ : 무위험수익률
- $\sigma$ : 주가 변동성
- $T$ : 전체 기간
- $Z$ : 정규분포로부터 나온 난수

In [ ]:
# T 시점 후의 가격 계산 함수
def terminal_price(S_0, mu, sigma, Z, T):
    return S_0 * np.exp((mu - sigma**2 / 2) * T + sigma * np.sqrt(T) * Z)

In [ ]:
# 자산간 상관계수를 고려한 난수 생성
Z = np.matmul(L, norm.rvs(size=[len(data.columns), n_simulations]))

# 주가 미래 가격에 대한 시뮬레이션
price_future = terminal_price(np.array(S_0).reshape(len(S_0), 1), 0, np.array(sigma).reshape(len(sigma), 1), Z, T)

In [ ]:
# 포트폴리오 시뮬레이션
portval_future = np.sum(price_future, axis=0)

# 포트폴리오 수익률 계산
port_return = (portval_future - portval_current) / portval_current

In [ ]:
type(port_return)

### 몬테카를로 시뮬레이션 VaR와 CVaR 계산

In [ ]:
# 포트폴리오 VaR
port_VaR = pd.Series(port_return).quantile(delta)

# 포트폴리오 CVaR
port_CVaR = np.mean(port_return[port_return <= port_VaR])

In [ ]:
# VaR, CVaR 그래프
plt.figure(figsize=(10, 8))
plt.hist(port_return, bins=100)
plt.title('Monte Carlo Simulation')
plt.axvline(port_VaR, color='g', linestyle='dashed', label='VaR')
plt.axvline(port_CVaR, color='r', linestyle='dashed', label='CVaR')
plt.legend()

Copyright 2022. 퀀트대디. All rights reserved.